In [1]:
import scipy as sp
import collections
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
# from scipy.spatial import distance_matrix
# import geopandas as gpd
# import shapely.wkt
# from shapely.geometry import Point
# import json
# import sklearn
# from sklearn.neighbors import NearestNeighbors
# import sklearn.preprocessing as preprocessing
# from shapely.geometry import Point
# from shapely.geometry.polygon import Polygon


In [2]:
# Input a pandas series 
def shannon(data):
    p_data= data.value_counts()/(len(data))*1.0 # calculates the probabilities
    entropy=sc.stats.entropy(p_data)  # input probabilities to get the entropy 
    return entropy